# KCMC Problem

We have a set of points of interest (POIs) that are static locations in space that continuously produce information. To gather that information, the POIs must be monitored by SENSORS.

Sensors have a limited "sensing range", so often a sensor cannot monitor a POI because it is too far away, "out of range". When a POI is monitored by a sensor, it must be within the sensing range of the sensor, and we say that the POI is "covered" by the sensor. A single sensor can monitor all the POIs that are within its sensing range.

We have several static, immovable locations in space in which sensors can be installed. Those locations are often called "installation spots". A single sensor may be installed in each spot.

Often a single sensor cannot monitor a POI to satisfaction (a condition called "insufficient quality of service"), so each POI must be covered by at least $\dot{K}$ sensors. The number of sensors that cover a POI is called its "coverage".

Sensors are not designed to store or process the data collected. Instead, each sensor must transmit the data it has collected to other sensors. Those other sensors may re-transmit the data to other sensors, until it reaches a special sensor called SINK. A sink is a sensor capable of storing or processing data. We often cannot choose where the sinks are installed.

Sensors are prone to failure, so we ensure that the data from every POI is continually transmitted to the sink by guaranteeing that we have at least $\dot{M}$ different sets of sensors capable of transmitting the data from each POI to a sink. Those sets must be disjunct - no sensor may belong to more than one set. Disjunct sets guarantee that even if failures in a set render it unable to get data from a POI to any sink, the sensors in other sets can deliver it. The number of disjunct sets of sensors that connect a POI to a sink is called its "connectivity".

Sensors have a cost, so we want to install as few as possible while still guaranteeing that each POI is covered with at least $\dot{K}$ different sensors and the data form each POI have at least $\dot{M}$ different paths to a sink. This is the $\dot{K}$-Coverage $\dot{M}$-Connectivity problem.

We represent a instance of the KCMC problem as a [graphs](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)). In discrete mathematics, a graph is an structure formed by "nodes" or "vertices" (usually represented as circles or dots) distributead in any way in the space, since the exact position of each vertice is not relevant to the graph. Instead, the connections between nodes (the edges) is the information we are concerned with.

In our graph representation, each node in the graph may be:
- installation spot for a sensor
- a POI
- a sink

If a node represents a POI, it is connected to all installation spots where a sensors would be able to cover the POI.
If the node represents a sink, it is connected to all instalation spots where sensors would be capable of transmitting data to the sink.
If the node represent an installation spot for a sensor, it is connected to the POIs an installed sensor would able to cover, and all the sinks and other sensor installation spots that a sensor placed at the current spot would be able to communicate with.

Our strategy to solve the KCMC problem is to find $\dot{M}$ sets of sensor installation spots that form [Steiner trees](https://en.wikipedia.org/wiki/Steiner_tree_problem). Steiner trees are sub-graphs that connect a subset of nodes (the POIs and sinks) using as few as possible nodes that are not in the subset.
The resulting $\dot{M}$ minimal disjunct Steiner Trees contain all the sensor instalation spots we require to guarantee that each POI in the KCMC instance has at least $\dot{M}$ connectivity.
Then we simply add as few as possible sensor installation spots so that each POI has at least $\dot{K}$ coverage.

A more interesting way to describe our strategy is using an integer-linear programming [(ILP) formulation](https://en.wikipedia.org/wiki/Integer_programming), in which we use discrete linear algebra equations and inequations to describe the elements and constraints of the KCMC problem.
ILP formulations are an standardized way of describing problems, and have many useful mathematical properties.

In this notebook, we explore our ILP formulation for the KCMC problem and implement it to obtain solutions. 

## Imports & Data gathering

The code below imports the [GUROBI](https://en.wikipedia.org/wiki/Gurobi) Optimizer, a proprietary [Solver](https://en.wikipedia.org/wiki/Solver) software capable of quickly finding the best possible solution to any problem expressed as a valid ILP formulation, given small enough instances.

The KCMC_Instance object imported is a custom python facility that reads a given KCMC instance and readily exposes its attributes.

In [ ]:
import pandas as pd
instances = pd.read_csv('/data/small_instances.csv', sep='|', header=None)
instances.columns = ['serial', 'kcmc']

INSTANCE_ROW = 2
DRY_RUN = False

from kcmc_instance import KCMC_Instance
kcmc_instance = KCMC_Instance(instances.iloc[INSTANCE_ROW]['serial'], True, True, False)
kcmc_k = int(instances.iloc[INSTANCE_ROW]['kcmc'][2])
kcmc_m = int(instances.iloc[INSTANCE_ROW]['kcmc'][4])

import gurobipy as gp
from gurobipy import GRB

(kcmc_k, kcmc_m), kcmc_instance

The code below imports an instance of the KCMC problem and exposes its attributes as several variables relevant to the KCMC problem:

- P: The set of POIs in the instance
- I: The set of sensors in the instance
- s: The ONLY sink of the instance

- A_c: The set of "covering edges", each a tuple where the first position holds a POI and the second a sensor that covers the POI
- A_s: The set of "sink edges", each a tuple where the first position holds a sensor connected to the sink and the second contains the sink
- A_g: The set og "graph edges", each a tuple where the first position holds a sensor and the second holds another sensor connected to the first.
- A: The complete set of edges, A_c+A_s+A_g

Some other useful data is also readied for processing:

- iC: The "Inverse Coverage Graph" - a mapping of each POI to the set of sensors that cover the POI.
- L: A sequence containing all integer values between 0 and $\dot{M}$

In [ ]:
P = kcmc_instance.pois
I = kcmc_instance.sensors
s = list(kcmc_instance.sinks)[0]  # HARD-CODED ASSUMPTION OF SINGLE-SINK!

A_c = kcmc_instance.poi_edges
A_s = kcmc_instance.sink_edges
A_g = kcmc_instance.sensor_edges
A = A_c + A_g + A_s

iC = kcmc_instance.inverse_coverage_graph
L = [str(m) for m in range(kcmc_m)]

## Multi-Flow ILP

We have developed two different ILP formulations - Multi-Flow and Single-Flow.  
The Multi-Flow formulation clearly identifies the steiner tree to which each edge connecting used sensors belongs to, while the Single-Flow alternative does not.  
Each formulation has its own advantages and disadvantages that we must study.

### Variables

Variables are structures that store information about the KCMC problem.
A variable can be seen as an array of dimensions relative to the elements of the problem.

In this section, we describe and implement the two variables used in our Multi-Flow ILP formulation.
But first, we create a new GUROBI model object. It will print information about this copy of the GUROBI software.

Then, we add variables to the model.

In [ ]:
model_mf = 'KCMC Multi-Flow'
model_mf = gp.Model(model_mf)
model_mf.setParam(GRB.Param.Threads, 1)
model_mf

#### Variable $X_j^m$
This binary variable marks the usage of sensor $j \in V$ at the $m$-th Steiner tree ($m \in M$).

It can be seen as a 2-D array, the first dimension representing sensor placement spots and the second the $\dot{M}$ Steiner trees. Thus, we have $\dot{M} * |V|$ possible values for this variable.  
This variable is defined differently in our Multi-Flow and our Single-Flow ILPs.  
Due to a limitation in GUROBI, we use the $L$ sequence of $[0:\dot{M}[$ instead of the primitive value of $\dot{M}$.

In [ ]:
X_mf = model_mf.addVars(I, L, name='x', vtype=GRB.BINARY)

len(X_mf)

#### Variable $Y_{ji}^{km}$

This binary variable marks the usage of edge $(j,i) \in E$ at the $m$-th Steiner tree that connects POI $k \in P$ to the sink.

It can be seen as a 4-D array, the first two dimensions representing the elements in the edges of the graph, the third being a POI, and the the forth being the $\dot{M}$ Steiner trees.
Thus, we have $|E| * |P| * \dot{M}$ possible values for this variable.
This variable is defined differently in our Multi-Flow and our Single-Flow ILPs.  
Due to a limitation in GUROBI, we use the $L$ sequence of $[0:\dot{M}[$ instead of the primitive value of $\dot{M}$.

The Y variable can be allowed to assume real values instead of restricted to binary ones. It does not affect the quality of results and reduces the processing time of the model.

In [ ]:
#Y_mf = model_mf.addVars(A, P, L, name='y', vtype=GRB.BINARY)
Y_mf = model_mf.addVars(A, P, L, name='y')

len(Y_mf)

### Objective Function: MINIMIZE $\sum_{j \in V}\sum_{m \in M} x_{j}^{m} $

The objective function in an ILP formulation describes how the result of each intermediate solution will be evaluated. The objective of our optimization is to MINIMIZE the result of the objective function.

In [ ]:
model_mf.setObjective(X_mf.sum('*', '*'), GRB.MINIMIZE)

### Constraints

Constraints are rules limiting to the values that the variables described above may assume.
In this section we describe and implement the constraints.

#### Disjunction
#### Flow (POI)
#### Flow (Sensor)
#### Flow (Sink)
#### Projection
#### K-Coverage

In [ ]:
# Disjunction -----------------------------------------
disjunction_mf = model_mf.addConstrs(
    (X_mf.sum(i, '*') <= 1
     for i in I),
    name="disjunction"
)

# Flow ------------------------------------------------
flow_p_mf = model_mf.addConstrs(
    ((  gp.quicksum(Y_mf.select(p, '*', p, l))
      - gp.quicksum(Y_mf.select('*', p, p, l))) == 1
     for p in P
     for l in L),
    name="flow_p"
)

flow_i_mf = model_mf.addConstrs(
    ((  gp.quicksum(Y_mf.select(i, '*', p, l))
      - gp.quicksum(Y_mf.select('*', i, p, l))) == 0
     for i in I
     for p in P
     for l in L),
    name="flow_i"
)

flow_s_mf = model_mf.addConstrs(
    ((  gp.quicksum(Y_mf.select(s, '*', p, l))
      - gp.quicksum(Y_mf.select('*', s, p, l))) == -1
     for p in P
     for l in L),
    name="flow_s"
)

# Projection ------------------------------------------
projection_mf = model_mf.addConstrs(
    (Y_mf.sum(i, '*', p, l) <= X_mf.sum(i, l)
     for i in I
     for p in P
     for l in L),
    name="projection"
)

# K-Coverage ------------------------------------------
k_coverage_mf = model_mf.addConstrs(
    (gp.quicksum(X_mf.select(iC[p], '*')) >= kcmc_k
     for p in P),
    name="k_coverage"
)

### Runtime & PLOT

In [ ]:
model_mf.optimize()